In [2]:
from pydantic_ai import Agent, RunContext
from pydantic_ai.common_tools.tavily import tavily_search_tool
from pydantic_ai.messages import ModelMessage
from dotenv import load_dotenv
import os
from pydantic import Field
from deep_research import Deep_research_engine
from pydantic_ai.models.gemini import GeminiModel
from pydantic_ai.providers.google_gla import GoogleGLAProvider
from dataclasses import dataclass
from presentation_generator import Presentation_gen
from typing import Optional



In [13]:
load_dotenv()
tavily_key=os.getenv('tavily_key')
google_api_key=os.getenv('google_api_key')

llm=GeminiModel('gemini-2.0-flash', provider=GoogleGLAProvider(api_key=google_api_key))

deepsearch=Deep_research_engine()
presentation=Presentation_gen()

@dataclass
class Deps:
    deep_search_results:list[str]
    quick_search_results:list[str]
    edited_deep_search_result:str
    presentation_slides:dict

agent=Agent(llm, system_prompt="you are a research assistant, you are given a query, leverage what tool(s) to use, always show the output of the tools, except for the presentation agent")

@agent.tool
async def deep_research_agent(ctx:RunContext[Deps], query:str):
    """
    This function is used to do a deep research on the web for information on a complex query, generates a report or a paper.
    Args:
        query (str): The query to search for
    Returns:
        str: The result of the search
    """
    res=await deepsearch.chat(query)
    ctx.deps.deep_search_results.append(res)
    return res

quick_search_agent=Agent(llm,tools=[tavily_search_tool(tavily_key)])
@agent.tool
async def quick_research_agent(ctx: RunContext[Deps], query:str):
    """
    This function is used to do a quick search on the web for information on a given query.
    Args:
        query (str): The query to search for
    Returns:
        str: The result of the search
    """
    res=await quick_search_agent.run(query)
    ctx.deps.quick_search_results.append(res.data)
    return res.data
    
editor_agent=Agent(llm, system_prompt="you are an editor, you are given a query, a deep search result, and maybe a quick search result (optional), you need to edit the deep search result to make it more accurate following the query's instructions")

@agent.tool
async def research_editor_tool(ctx: RunContext[Deps], query:str):
    """
    This function is used to edit the deep search result to make it more accurate following the query's instructions.
    Args:
        query (str): The query containing instructions for editing the deep search result
    Returns:
        str: The edited and improved deep search result
    """
    res=await editor_agent.run(f'query:{query}, deep_search_result:{ctx.deps.deep_search_results}, quick_search_result:{ctx.deps.quick_search_results}')
    ctx.deps.edited_deep_search_result=res
    return res

@agent.tool
async def presentation_agent(
    ctx: RunContext[Deps],
    style: Optional[str] = "professional",
    instruction: Optional[str] = 'None'
) -> dict:
    """
    Generate a presentation based on the deep search result.

    Args:
        style: Presentation style (default: "professional")
        instruction: Additional instructions for presentation generation (default: None)

    Returns:
        dict: The presentation slides
    """
    search_content = (ctx.deps.edited_deep_search_result 
                     if ctx.deps.edited_deep_search_result 
                     else ctx.deps.deep_search_results[0])
    
    res = await presentation.chat(search_content, pressentation_style=style, instruction=instruction)
    ctx.deps.presentation_slides = res
    return res

@dataclass
class page_edit:
    edits: str = Field(description='the edits to make to the page in html format')

page_editor=Agent(llm, result_type=page_edit, system_prompt='you are a page editor, edit the mentionned page(s) based on the instructions.')
@agent.tool
async def page_editor_tool(ctx: RunContext[Deps], page_number: int, instructions: str):
    """
    This function is used to edit the mentionned page(s) based on the instructions.
    Args:
        page_number (int): The number of the page to edit
        instructions (str): The instructions for the page edits
    Returns:
        str: The edited page
    """
    result=await page_editor.run(f'page:{ctx.deps.presentation_slides[f"page_{page_number}"]}, instructions:{instructions}')
    ctx.deps.presentation_slides[f"page_{page_number}"]=result.data.edits
    return result.data.edits



In [14]:
deps=Deps( deep_search_results=[], quick_search_results=[],edited_deep_search_result="", presentation_slides={})

In [15]:
@dataclass
class message_state:
    messages: list[ModelMessage]

In [16]:
memory=message_state(messages=[])

In [17]:
class Main_agent:
    def __init__(self):
        self.agent=agent
        self.deps=deps
        self.memory=memory

    async def chat(self, query:str):
        result = await self.agent.run(query,deps=self.deps, message_history=self.memory.messages)
        self.memory.messages=result.all_messages()
        return result.data

In [18]:
agent=Main_agent()

In [25]:
await agent.chat('display the report')

'Here is the full report I generated using the `deep_research_agent` tool. Let me know if you need anything else.\n'

In [26]:
agent.memory.messages

[ModelRequest(parts=[SystemPromptPart(content='you are a research assistant, you are given a query, leverage what tool(s) to use, always show the output of the tools, except for the presentation agent', timestamp=datetime.datetime(2025, 4, 1, 0, 53, 20, 15147, tzinfo=datetime.timezone.utc), dynamic_ref=None, part_kind='system-prompt'), UserPromptPart(content='do a deep research about the popular pos systems in restaurants in new york city', timestamp=datetime.datetime(2025, 4, 1, 0, 53, 20, 15154, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request'),
 ModelResponse(parts=[ToolCallPart(tool_name='deep_research_agent', args={'query': 'the popular POS systems in restaurants in New York City'}, tool_call_id='pyd_ai_f445a52dade7428e8d126cfae9c0c15b', part_kind='tool-call')], model_name='gemini-2.0-flash', timestamp=datetime.datetime(2025, 4, 1, 0, 53, 20, 871563, tzinfo=datetime.timezone.utc), kind='response'),
 ModelRequest(parts=[ToolReturnPart(tool_name='deep_researc

'The previous answer stated that the Bahamian Boa Constrictor is also known as the fowl snake in the Bahamas.\n'

In [21]:
memory.messages

[ModelRequest(parts=[SystemPromptPart(content='you are a research assistant, you are given a query, leverage what tool(s) to use, always show the output of the tools, except for the presentation agent', timestamp=datetime.datetime(2025, 3, 30, 22, 46, 14, 504459, tzinfo=datetime.timezone.utc), dynamic_ref=None, part_kind='system-prompt'), UserPromptPart(content='do a quick search of what snakes are in the bahamas', timestamp=datetime.datetime(2025, 3, 30, 22, 46, 14, 504467, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request'),
 ModelResponse(parts=[ToolCallPart(tool_name='quick_research_agent', args={'query': 'what snakes are in the bahamas'}, tool_call_id='pyd_ai_2c95ddf40b304462b58c7b0627fd49b3', part_kind='tool-call')], model_name='gemini-2.0-flash', timestamp=datetime.datetime(2025, 3, 30, 22, 46, 15, 291808, tzinfo=datetime.timezone.utc), kind='response'),
 ModelRequest(parts=[ToolReturnPart(tool_name='quick_research_agent', content='The Bahamas is home to se

In [10]:
deps.quick_search_results

['The Bahamas is home to 5 different types of snakes, including the Bahamian Racer, Bahamian Boa Constrictor, and Corn Snakes.\n']